# Исследование надежности заемщиков


На исследование представлены данные банка со статистикой платежеспособности клиентов (в том числе факты просрочки).

Необходимо выяснить, как семейное положение, количество детей, уровень дохода и цели кредита влияют на факт его погашения в срок.

Результаты исследования будут представлены кредитному отделу банка для учета при построении модели кредитного скоринга.

Качество данных не высокое, поэтому большое внимание будет уделено их предобработке.



## Изучим общую информацию о данных

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/reliability_of_borrowers_data.csv')


In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Столбец `total_income` хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.


In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типа занятости `income_type`.

In [26]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [27]:
data.isna().sum()

children                 0
days_employed            0
dob_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
total_income_category    0
purpose_category         0
dtype: int64


Причина пропусков, возможно, в том, что у заемщиков нет соответствующих сведений (пенсионеры и безработные) или они не пожелали оставлять сведения. Также, возможно, пропуски вызваны техническим фактором, например были утрачены в результате действий с базой данных. Необходимо обратиться к ответственному сотруднику за разъяснениями.
Замена пропусков медианными значениями была корректней, так как большая разница  между значениями может исказить требуемый результат при выводе среднего значения.

### Обработка аномальных значений

Имеются отрицательные значения с количеством дней трудового стажа в столбце `days_employed`. Замените все отрицательные значения положительными.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Изучим медианное значение трудового стажа `days_employed` в днях для каждого типа занятости.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) аномально большие значения. Исправить такие значения не имеется возможности, поэтому оставим их как есть.

Выведем перечень уникальных значений столбца `children`.

In [38]:
data['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Удалим строки, в которых встречаются аномальные значения из датафрейма

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [28]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

В столбце `education` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [29]:
data['education'] = data['education'].str.lower()

In [30]:
data.duplicated().sum()

0

In [31]:
data = data.drop_duplicates()

### Категоризация данных

Создадим столбец `total_income_category` с категориями уровня доходов:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [32]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [33]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Создадим столбец `purpose_category` с категориями по целям получения кредита:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [34]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [35]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Анализ данных и проверка гипотез.

### Зависимость между количеством детей и возвратом кредита в срок

In [36]:
dependence = data.pivot_table(index = 'children', values = 'debt', aggfunc = {'count', 'sum'})
dependence = dependence.rename(columns = {'count':'all', 'sum':'debts'})
dependence['percent'] = round(dependence['debts'] / dependence['all'] * 100, 2)
dependence

,all,debts,percent
children,,,
0,14091,1063,7.54
1,4808,444,9.23
2,2052,194,9.45
3,330,27,8.18
4,41,4,9.76
5,9,0,0.00


Наличие детей влияет на возврат кредита в срок. Отсутствие детей увеличивает вероятность возврата кредита в срок почти на 2 %, если опираться на данные заемщиков без детей, с 1 и 2 детьми (так как по ним имеется больше значений: 14091, 4808 и 2052, соответственно). Расчет по заемщикам с 3 и 4 детьми менее показателен, так как данных значительно меньше (330 и 41), тем не менее он так же отражает большую вероятность просрочки заемщиками с детьми. Заемщиков с 5 детьми всего 9, и просрочек у них нет, но данных слишком мало для однозначного вывода.

### Зависимость между семейным положением и возвратом кредита в срок

In [23]:
dependence = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = {'count', 'sum'})
dependence = dependence.rename(columns = {'count':'all', 'sum':'debts'})
dependence['percent'] = round(dependence['debts'] / dependence['all'] * 100, 2)
dependence

,all,debts,percent
family_status,,,
Не женат / не замужем,2796,273,9.76
в разводе,1189,84,7.06
вдовец / вдова,951,63,6.62
гражданский брак,4134,385,9.31
женат / замужем,12261,927,7.56


Семейное положение влияет на возврат кредита в срок. Заемщкики, которые не состоят и не состояли в зарегистрированном браке (не женаты / не замужем, в гражданском браке) на 2-2,5 % чаще допускают просрочки. Наименьшая вероятность просрочки у овдовевших и разведенных.

### Зависимость между уровнем дохода и возвратом кредита в срок

In [24]:
dependence = data.groupby('total_income_category').agg({'debt': ['count', 'sum']})
dependence = dependence.rename(columns = {'count':'all', 'sum':'debts'})
dependence['percent'] = round(dependence['debt']['debts'] / dependence['debt']['all'] * 100, 2)
dependence

debt       percent
                         all debts        
total_income_category                     
A                         25     2    8.00
B                       5014   354    7.06
C                      15921  1353    8.50
D                        349    21    6.02
E                         22     2    9.09

Уровень дохода не влияет на возврат кредита в срок. Не прослеживается закономерность. В расчет не берется максимальный и минимальный доход (A и E) ввиду малого количества данных (25 и 22 строки). Процент просрочек может быть относительно схожим как у низкого дохода (D - 6%), так и у высокого (B - 7 %), в то время как у среднего он самый высокий (C - 8,5%) .

### Влияние целей кредита на его возврат в срок

In [25]:
dependence = data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = {'count', 'sum'})
dependence.columns = ['all', 'debts']
dependence['percent'] = round(dependence['debts'] / dependence['all'] * 100, 2)
dependence

,all,debts,percent
purpose_category,,,
операции с автомобилем,4279,400,9.35
операции с недвижимостью,10751,780,7.26
получение образования,3988,369,9.25
проведение свадьбы,2313,183,7.91


Цели кредита влияют на его возврат в срок. Так, при более "глобальных" целях: на недвижимость и свадьбу, процент просрочек ниже до 2%, нежели при операциях с авто и получением образования.

## Выводы

На исследование представлена база данных заемщиков, с целью определения влияния на возврат кредита в срок следующих факторов:
- семейное положение;
- наличие детей;
- уровень дохода;
- цели кредита.

В базе данных обнаружился ряд проблем, затрудняющих проведенеие исследования (пропущенные и аномальные значения, дубликаты). В связи с этим дополнительно была выполнена предобработка данных (замена пропущенных значений медианными, отрицательных - положительными, удаление явных дубликатов, изменение разных написаний схожих значений и удаление лишних).

После проведения предобработки было осуществлено исследование. Принцип исследования заключался в распределении общей массы заемщиков по соответствующим категориям (вышеуказанным факторам), нахождение среди них доли заемщиков с просроченной задолженностью, и сравнение результатов. Для анализа влияния уровня дохода и цели кредита на его возврат в срок были созданы дополнительные столбцы: total_income_category и purpose_category с распределением исходных данных по соответствующим категориям.

В результате исследования были сделаны следующие выводы:
- Отсутствие детей повышает вероятность возврата кредита в срок на 2 %.
- Заемщики, которые состоят или состояли в браке чаще возвращают кредит в срок на 2 - 2,5 %.
- Кредит, который берут для операций с недвжмостью и для свадьбы, чаще возвращают в срок на 2 %.
- Уровень дохода не влияет на возврат кредита в срок.

Рекомендации:
- Учитывать результаты настоящего исследования при осуществлении скоринга.
- Разработать инструкцию по заполнению базы данных довести её до ответственных за работу с ней сотрудников с целью минимизации проблем в данных (пропуски, дубликаты, неединообразное написание, ошибочные значения)
